In [1]:
"""
① (10점) 주어진 5개의 data set (Am241_countrate.csv, Ba133_countrate.csv , Cs137_countrate.csv, 
Eu152_countrate.csv , Background_countrate.csv 을 각각 읽어 들이시오.
- 각 file 첫번째 열(column) 은 ADC Channel 번호, 두번째 열 은 count rate (#/s)
"""
from scipy.optimize import curve_fit
from scipy.stats import linregress
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

colnames = ['ADC Channel Number', 'count rate (#/s)']
Am241 = pd.read_csv('기말프로젝트_압축/Am241_countrate.csv', names = colnames, header=None)
Ba133 = pd.read_csv('기말프로젝트_압축/Ba133_countrate.csv',names = colnames, header=None)
Cs137=pd.read_csv('기말프로젝트_압축/Cs137_countrate.csv',names = colnames, header=None)
Eu152=pd.read_csv('기말프로젝트_압축/Eu152_countrate.csv',names = colnames, header=None)
Background = pd.read_csv('기말프로젝트_압축/Background_countrate.csv',names = colnames, header=None)

In [2]:
"""
② (10점) 이제 각각의 file에서 자료 count rate의 불확실성 (uncertainty)를 구하려 한다. 검출기의
count가 Poisson 분포를 따른다고 가정하고 count rate 의 불확실성을 구한 후 읽어 들인 5개
의 dataset의 3번째 열에 계산 값을 추가하시오.

"""

Am241['uncertainties']=np.sqrt(Am241['count rate (#/s)']*3600)/3600
Ba133['uncertainties']=np.sqrt(Ba133['count rate (#/s)']*28252)/28252
Cs137['uncertainties']=np.sqrt(Cs137['count rate (#/s)']*22880)/22880
Eu152['uncertainties']=np.sqrt(Eu152['count rate (#/s)']*23352)/23352
Background['uncertainties']=np.sqrt(Background['count rate (#/s)']*4328)/4328

In [3]:
""""3_1_Linear Least Square Fit_Spring.ipynb"③ (10점) 방사선원을 사용하여 획득한 4개 자료의 2번째 열에 기록된 값은 방사선원과 배경의
영향이 혼합되어 있다고 추정된다 (𝑪𝒊𝒔𝒐𝒕𝒐𝒑𝒆 + 𝒃𝒂𝒄𝒌𝒈𝒐𝒓𝒖𝒏𝒅). 4개 자료의 2번째 열에서 background 
coutrate 을 (𝑪𝒃𝒂𝒄𝒌𝒈𝒓𝒐𝒖𝒏𝒅) 제거하여 4번째 column에 삽입하라
"""

Am241['count rate wo background (#/s)'] = Am241['count rate (#/s)']- Background['count rate (#/s)']
isotopes = [Ba133, Cs137, Eu152]
for i in isotopes:
    bccount1 = i['count rate (#/s)'][(i['ADC Channel Number']<255)]- Background['count rate (#/s)']
    bccount2 = i['count rate (#/s)'][(i['ADC Channel Number']>=255)]
    i['count rate wo background (#/s)']= pd.concat([bccount1, bccount2])

      ADC Channel Number  count rate (#/s)  uncertainties  \
0                      0          0.000359       0.000113   
1                      1          0.000040       0.000038   
2                      2          0.000160       0.000075   
3                      3          0.000200       0.000084   
4                      4          0.000320       0.000106   
...                  ...               ...            ...   
1015                1015          0.000000       0.000000   
1016                1016          0.000000       0.000000   
1017                1017          0.000000       0.000000   
1018                1018          0.000069       0.000049   
1019                1019          0.000035       0.000035   

      count rate wo background (#/s)  
0                          -0.000334  
1                           0.000040  
2                          -0.000302  
3                          -0.000262  
4                          -0.000143  
...                              